In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_mode = False

In [3]:
path = "/mnt/cephfs/ml_data/TAO_detsim_J22/"
data = pd.read_csv(f'{path}processed_data/ProcessedTrain/ProcessedTrain.csv.gz')

In [4]:
FC_cut = 0.65
N = int(1.15e6)

data = data.reset_index(drop=True)
data = data[data['edepR'] < FC_cut]
data = data[:N]
data.head()

,AccumCharge,nPMTs,R_cc,rho_cc,x_cc,y_cc,z_cc,gamma_z_cc,gamma_y_cc,gamma_x_cc,...,pe_75p,pe_80p,pe_85p,pe_90p,pe_95p,edep,edepX,edepY,edepZ,edepR
1,20058.0,3751.0,0.413871,0.160509,-0.033655,-0.156941,-0.381479,-2.376677,-0.409811,-0.081587,...,6.0,7.0,9.0,12.0,18.0,4.968661,-0.047378,-0.223696,-0.574465,0.618300
3,23914.0,3885.0,0.358204,0.108909,0.107540,0.017212,-0.341246,-3.133318,0.048106,0.314740,...,7.0,9.0,10.0,13.0,18.0,5.843454,0.147694,0.022231,-0.516246,0.537417
6,25500.0,3905.0,0.374247,0.301253,0.250178,0.167824,0.222052,0.737094,0.501702,0.898828,...,8.0,9.0,11.0,14.0,21.0,6.099024,0.347853,0.236193,0.326620,0.532419
8,33640.0,4018.0,0.188545,0.024454,0.019590,-0.014637,-0.186953,-7.645026,-0.077864,0.104468,...,10.0,11.0,12.0,14.0,18.0,8.350126,0.025446,-0.010741,-0.293094,0.294393
9,11417.0,3677.0,0.191311,0.103964,-0.079645,-0.066822,-0.160597,-1.544738,-0.372763,-0.457876,...,4.0,4.0,5.0,6.0,7.0,2.929927,-0.104180,-0.106995,-0.238049,0.281013


In [5]:
from sklearn.metrics import mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
size = int(1e6)
n_feats = len(data.columns) - 5

X_val = data.iloc[:, :-5][size:]
y_val = data.iloc[:, -5][size:]
data = data[:size]

In [7]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

if train_mode:    
    n_folds = 5
    rmse_scores = []
    mape_scores = []
    
    kfold = KFold(n_folds, True, random_state=22)    
    for train, test in tqdm(kfold.split(data), "Folds... ", leave=False):        
        xgbreg = XGBRegressor(
                max_depth=9,
                learning_rate=0.08,
                n_estimators=10000,
#                 tree_method='gpu_hist',
#                 gpu_id=0,
        )
                            
        X_train = np.array(data)[train][:, :n_feats]
        y_train = np.array(data)[train][:, n_feats]
                            
        X_test = np.array(data)[test][:, :n_feats]
        y_test = np.array(data)[test][:, n_feats]

        xgbreg.fit(X_train, y_train,
                   verbose=False,
                   eval_set=[(np.array(X_val), np.array(y_val))],
                   early_stopping_rounds=3)
        
        y_predict = xgbreg.predict(X_test)
        rmse = mean_squared_error(y_test, y_predict)**0.5
        mape = mean_absolute_percentage_error(y_test, y_predict)
        rmse_scores.append(rmse)
        mape_scores.append(mape)
    
    result = np.array([[np.mean(mape_scores), np.std(mape_scores)], [np.mean(rmse_scores), np.std(rmse_scores)]])
    np.savez_compressed('feature_selection/all_features_metrics.npz', a=result)

In [8]:
all_features_metrics = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a']
all_features_metrics

array([[8.47010322e-01, 2.09017053e-03],
       [5.56608926e-02, 9.59862877e-05]])

In [ ]:
all_features_metric = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][0][0]
eps = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][0][1]

opt_features = ['AccumCharge', 'rho_cc']#[]
current_metrics = [1.2568856799038168, 0.9518705976467909]#[]
current_metric_stds = [0.003695801053828588, 0.004235039357730335]#[]
current_metric = 100

features = data.iloc[:, :-5].columns
features = features.drop(opt_features)
while True:#abs(all_features_metric - current_metric) > eps:
    metrics = []
    metric_stds = []
    for feature in tqdm(features, "Features loop"):

        X = data.iloc[:, :-5][opt_features+[feature]]
        y = data.iloc[:, -5]

        xgbreg = XGBRegressor(
            max_depth=9,
            learning_rate=0.08,
            n_estimators=10000,
            random_state=22,
#             tree_method='gpu_exact',
#             gpu_id=0,
        )

        scores = cross_val_score(
            xgbreg,
            X,
            y,
            cv=5,
            n_jobs=-1,
            verbose=False,
            fit_params={
                'eval_set': [(X_val[opt_features+[feature]], y_val)],
                'early_stopping_rounds':3
            },
            scoring='neg_mean_absolute_percentage_error'
        )

        metric = -100*scores.mean()
        metric_std = (100*scores).std()
        metrics.append(metric)
        metric_stds.append(metric_std)
      
    best_metric_ind = np.argmin(metrics)
    current_metric = metrics[best_metric_ind]
    current_metrics.append(current_metric)

    current_metric_std = metric_stds[best_metric_ind]
    current_metric_stds.append(current_metric_std)

    opt_features.append(features[best_metric_ind])
    features = features.drop(features[best_metric_ind])

    print(current_metrics)
    print(current_metric_stds)
    print(opt_features)

    np.savez_compressed('feature_selection/opt_features.npz', a=np.array(opt_features))
    np.savez_compressed('feature_selection/current_metrics.npz', a=np.array(current_metrics))
    np.savez_compressed('feature_selection/current_metric_stds.npz', a=np.array(current_metric_stds))

Features loop:   0%|          | 0/89 [00:00<?, ?it/s]

[1.2568856799038168, 0.9518705976467909, 0.876074944289119]
[0.003695801053828588, 0.004235039357730335, 0.00282086117549013]
['AccumCharge', 'rho_cc', 'ht_60p']


Features loop:   0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
np.savez_compressed('feature_selection/opt_features.npz', a=np.array(opt_features))
np.savez_compressed('feature_selection/current_metrics.npz', a=np.array(current_metrics))
np.savez_compressed('feature_selection/current_metric_stds.npz', a=np.array(current_metric_stds))

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'

current_metrics = np.load('feature_selection/current_metrics.npz', allow_pickle=True)['a']
opt_features = np.load('feature_selection/opt_features.npz', allow_pickle=True)['a']
current_metric_stds = np.load('feature_selection/current_metric_stds.npz', allow_pickle=True)['a']
all_features_metric = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][0][0]
eps = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][0][1]

all_features_metrics_lower = all_features_metric - eps
all_features_metrics_upper = all_features_metric + eps

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=opt_features,
        y=current_metrics,
        error_y=dict(
            type='data',
            width=10,
            array=current_metric_stds
        )
    )
)

fig.add_hline(
    y=all_features_metric,
    line=dict(dash='dash')
)

fig.add_hrect(
    y0=all_features_metrics_lower, 
    y1=all_features_metrics_upper,
    fillcolor="darkred",
    line_width=1,
    opacity=0.25,
)

fig.update_yaxes(title='MAPE, %')
fig.update_xaxes(title='Added feature')

fig.update_layout(

    xaxis = dict(
        showline=True,
        ticks='outside',
        mirror=True,
        linecolor='black',
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
    ),

    yaxis = dict(
        showline=True,
        ticks='outside',
        mirror=True,
        linecolor='black',
        tick0=0,
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
        zeroline=True,
        zerolinecolor='black',
        zerolinewidth=0.25
    ),
    
    font=dict(
        family="Times New Roman",
        size=16,
        color="Black"
    )
)

fig.show()